In [1]:
import os
import sys
sys.path.insert(0, '../../')

In [2]:
import gym
import numpy as np

Mujoco Wrappers test

In [3]:
from synchronized_wrapper.mujoco_wrapper import mujoco_env

In [4]:
game_id = 'HumanoidStandup-v2'
stack = 5
env = mujoco_env(game_id, stack=stack)
gym_env = gym.make(game_id)

print('action_space')
print(env.action_space.high[0], env.action_space.low[0], env.action_space.shape)
print(gym_env.action_space.high[0], gym_env.action_space.low[0], gym_env.action_space.shape) 

print('observation_space')
print(env.observation_space.high[0], env.observation_space.low[0], env.observation_space.shape)
print(gym_env.observation_space.high[0], gym_env.observation_space.low[0], gym_env.observation_space.shape)
assert env.observation_space.shape[0] == stack * gym_env.observation_space.shape[0]

assert env.observation_space.contains(env.reset())
while True:
    s, r, d, _ = env.step(np.random.uniform(low=-1.0, high=1.0, size=env.action_space.shape[0]))
    assert env.observation_space.contains(s)
    if d:
        break

env.close()
gym_env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
action_space
1.0 -1.0 (17,)
0.4 -0.4 (17,)
observation_space
inf -inf (1880,)
inf -inf (376,)


Atari Wrappers test (just test for RewardClipEnv)

In [5]:
from synchronized_wrapper.atari_wrapper import atari_env

In [6]:
game_id = 'BeamRiderNoFrameskip-v4'
env = atari_env(game_id)

allowed_actions = list(range(env.action_space.n))

env.reset()
reward = 0
real_reward = 0
while True:
    s, r, d, info = env.step(np.random.choice(allowed_actions))
    
    assert 'was_real_done' in info
    assert 'real_reward' in info
    assert r in [-1.0, 0.0, 1.0]
    
    reward += r
    real_reward += info['real_reward']
    if d:
        print(info)
        if info['was_real_done']:
            print(reward, real_reward)
            break

env.close()

{'real_reward': 0.0, 'was_real_done': False, 'ale.lives': 2}
{'real_reward': 0.0, 'was_real_done': False, 'ale.lives': 1}
{'real_reward': 0.0, 'was_real_done': True, 'ale.lives': 0}
10.0 440.0


Agent test

In [7]:
from synchronized_wrapper.atari_wrapper import get_atari_env_fn
from synchronized_wrapper.mujoco_wrapper import get_mujoco_env_fn
from synchronized_wrapper.agent import Agent

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:65277 to respond...
Waiting for redis server at 127.0.0.1:24875 to respond...
Starting local scheduler with the following resources: {'CPU': 32, 'GPU': 4}.

View the web UI at http://localhost:8894/notebooks/ray_ui15175.ipynb?token=a5150d899d1f5256c75a92855ba0496473ebc55cd9ae48f2



In [11]:
env_fn = get_atari_env_fn('BeamRiderNoFrameskip-v4')
num_agents = 8

env = Agent(num_agents=num_agents, env_fn=env_fn, basename='test')

print('action_n: ', env.action_space.n)

print('observation_space')
assert env.observation_space.shape == (num_agents, 84, 84, 4)
print(env.observation_space.high[0,0,0,0], env.observation_space.low[0,0,0,0], env.observation_space.shape, env.observation_space.dtype)

assert env.reset().shape == (num_agents, 84, 84, 4)

env.close()
# env = Agent(num_agents=num_agents, env_fn=env_fn, basename='test')
# env.close()

All subagents are ready! 
action_n:  9
observation_space
1.0 0.0 (8, 84, 84, 4) float32


In [9]:
np.repeat([np.array([1,2])], 4, axis=0)

array([[1, 2],
       [1, 2],
       [1, 2],
       [1, 2]])